In [174]:
import pandas as pd
import re

### Функция, которая проходя по датасету снизу вверх, создает новый готовый датасет с актуальными данными без пропусков пред. значений, если они были

In [175]:
def getFinalDataset(df):
    finalDf = pd.DataFrame(columns=df.columns)
    uidList = []
    for i in range(len(df)-1, -1, -1):
        row = df.iloc[i]
        curUid = row['uid']
        if curUid not in uidList:
            uidList.append(curUid)
            finalDf.loc[len(finalDf)] = list(df.iloc[i])
        else:
            a = finalDf.loc[(finalDf['uid'] == curUid)].fillna('-').squeeze(axis=0)
            idx = finalDf.loc[(finalDf['uid'] == curUid)].index
            for col in df.columns:
                if (a[col] == '-' and row[col] != '-') or (a[col] == 0 and row[col] != 0):
                    finalDf.loc[idx, col] = row[col]
    return finalDf


### Функция, вычленяющая ссылки из строки

In [176]:
def getClearSocial(string):
    pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    links = str(re.findall(pattern, string))
    if links == '[]':
        return '-'
    return links

In [177]:
pf1 = pd.read_excel('hotels_parsing_result.xlsx', sheet_name='platform1').fillna('-')
pf2 = pd.read_excel('hotels_parsing_result.xlsx', sheet_name='platfrom2').fillna('-')


### Получаем ссылки на соц.сети и очищаем описание отелей от гипертекстовой разметки

In [178]:
finalPf1 = getFinalDataset(pf1)
finalPf2 = getFinalDataset(pf2)

finalPf1['contact_social'] = finalPf1['contact_social'].apply(lambda x : getClearSocial(x))
finalPf2['contact_social'] = finalPf2['contact_social'].apply(lambda x : getClearSocial(x))

finalPf1['description'] = finalPf1['description'].apply(lambda x : x.replace('<br />', '').replace('•&nbsp;', ''))
finalPf2['description'] = finalPf2['description'].apply(lambda x : x.replace('<br />', '').replace('•&nbsp;', ''))

### Создаем дополнительную колонку, содержащую все способы связи/получения информации

In [179]:
finalPf2['allContacts'] = ''
for i in range(len(finalPf2)):
    tup = (finalPf2.loc[i, 'email'], finalPf2.loc[i, 'phone'], finalPf2.loc[i, 'website'])
    finalPf2.at[i, 'allContacts'] = tup

finalPf1['allContacts'] = ''
for i in range(len(finalPf1)):
    tup = (finalPf2.loc[i, 'email'], finalPf1.loc[i, 'phone'], finalPf1.loc[i, 'website'])
    finalPf1.at[i, 'allContacts'] = tup

### Топ-10 отелей по количеству телефонов, Platform2

In [180]:
for i in range(len(finalPf2)):
    finalPf2.loc[i, 'phoneCount'] = len(finalPf2.loc[i, 'phone'].split(','))
finalPf2.sort_values('phoneCount', ascending=False)[['title', 'phoneCount']].head(10)


,title,phoneCount
95,Goldy Land,6.0
113,"Панорама, мини-отель",6.0
48,"Ивкор, база отдыха",5.0
34,"Берлога на озере Калкан, база отдыха",5.0
449,"Сказка, мини-отель",4.0
170,"Vgosti, гостиница",4.0
347,"Межгорье, мини-гостиница",4.0
389,"Альпин Шале, гостевой дом",4.0
406,"La Bereg, гостевой комплекс",4.0
492,"Добр здравич, гостиница",4.0


### Топ-10 отелей по количеству отзывов, Platform2

In [181]:
finalPf2['review_count'] = finalPf2['review_count'].replace('-', 0)
finalPf2.sort_values('review_count', ascending=False)[['title', 'review_count']].head(10)

,title,review_count
499,"Hilton Garden Inn Ufa Riverside, отель",823.0
502,"Башкирия, гостиничный комплекс",780.0
496,"Hampton by hilton Ufa, отель",451.0
497,"Урал тау, гостиничный комплекс",416.0
514,AZIMUT Сити Отель Уфа,386.0
537,"Амакс Сити-отель, гостиница",358.0
500,"Президент отель, гостиничный комплекс",341.0
501,"Sheratonplaza Ufa Congress Hotel, гостиница",240.0
488,"ВолнаСити, отель",232.0
520,"Свояк, гостиничный комплекс",222.0


### Топ-10 отелей по количеству телефонов, Platform1

In [182]:
for i in range(len(finalPf1)):
    finalPf1.loc[i, 'phoneCount'] = len(finalPf1.loc[i, 'phone'].split(','))
finalPf1.sort_values('phoneCount', ascending=False)[['title', 'phoneCount']].head(10)

,title,phoneCount
0,Airport Hotel Ufa,1.0
3678,"Квартира ""Балтийские ночи"" в Самом Центре с Кл...",1.0
3686,Апартаменты Апарт Елена,1.0
3685,Апартаменты на Монетном Переулке,1.0
3684,Хостел КёнигLoft,1.0
3683,Квартира Пролетарская,1.0
3682,Мини-отель Robertina,1.0
3681,Квартира У Резиденции Королей,1.0
3680,Апартаменты в центре города на Нижнем озере,1.0
3679,Апартаменты Сергеева 5,1.0


### Топ-10 отелей по количеству отзывов, Platform1

In [183]:
finalPf1['review_count'] = finalPf1['review_count'].replace('-', 0)
finalPf1.sort_values('review_count', ascending=False)[['title', 'review_count']].head(10)

,title,review_count
3780,Radisson Blu Hotel Kaliningrad,1201
5513,AZIMUT Сити Отель Уфа,988
3695,Отель Кайзерхоф (Kaiserhof),930
36,Отель Ibis Калининград Центр,837
5516,Hilton Garden Inn Ufa Riverside,616
5510,Отель АМАКС Сити-отель,540
5515,Отель Hampton by Hilton Уфа,460
5328,Гостиничный Комплекс Президент Отель,414
39,Гостиница Мартон Палас Калининград,334
3943,"Гостиница ""Шкиперская""",302
